In [21]:
import numpy as np
from skimage.io import imread
import time
import platform
import tempfile
import os
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
import skimage
from skimage import io as skio
import IPython
from skimage.transform import rescale
from skimage.io import imread
from scipy.spatial import distance
import sys
from scipy.fft import dct, idct, dctn, idctn

In [37]:
#================================== Parameters initialization ==================================

sigma = 25 # variance of the noise

lamb2d = 2.0

lamb3d = 2.7

Step1_ThreDist = 2500 # threshold distance

Step1_MaxMatch = 16 # max matched blocks

Step1_BlockSize = 8

In [33]:
def noisegauss(im,br=sigma):
    """ Cette fonction ajoute un bruit blanc gaussier d'ecart type br
       a l'image im et renvoie le resultat"""
    imt=np.float32(im.copy())
    sh=imt.shape
    bruit=br*np.random.randn(*sh)
    imt=imt+bruit
    return imt

def viewimage(im, normalize=True,z=1,order=0,titre='',displayfilename=False):
    imin=im.copy().astype(np.float32)
    imin = rescale(imin, z, order=order)
    if normalize:
        imin-=imin.min()
        if imin.max()>0:
            imin/=imin.max()
    else:
        imin=imin.clip(0,255)/255 
    imin=(imin*255).astype(np.uint8)
    filename=tempfile.mktemp(titre+'.png')
    if displayfilename:
        print (filename)
    plt.imsave(filename, imin, cmap='gray')
    IPython.display.display(IPython.display.Image(filename))

In [34]:
# Step 1

def distpatch(patch1, patch2, khard=8):
    """
    Calcul distance entre deux patchs (sans seuillage).

    patch1, patch2: 1D arrays (flatten)
    """
    if patch1.size != patch2.size:
        print('Error: two patches should be in same size in function distpatch().\n')
        sys.exit()

    p1 = patch1.astype(np.float32)
    p2 = patch2.astype(np.float32)

    diff = p1 - p2
    dist = np.sqrt(np.sum(diff * diff)) / (khard ** 2)  # Normalization
    return dist

def Step1_grouping(im, center_i=None, center_j=None,
             patch_size=Step1_BlockSize, search_window=Step2_WindowSize,
             max_similar_patches=Step1_MaxMatch, threshard=Step1_ThreDist):
    """
    Find and group patches from the image *im*, return a 3D array of patches.

    Note that the distance computing is chosen from original paper rather than the analysis one.

    - im: 2D noised image
    - (center_i, center_j): Center of reference patch. If center = None, the center of the image will be used
    - patch_size: Size of patch (default = 8)
    - search_window: Size of the search window (in pixels)
    - max_similar_patches: Nhard
    - threshard: Distance threshold for accepting a patch
    """
    H, W = im.shape
    half_p = patch_size // 2
    half_w = search_window // 2

    # 1. Select the center of the reference patch.
    if center_i is None:
        center_i = H // 2
    if center_j is None:
        center_j = W // 2
    center_i = np.clip(center_i, half_p, H - half_p - 1)
    center_j = np.clip(center_j, half_p, W - half_p - 1)

    # 2. build the reference patch.
    ref_patch = im[center_i - half_p:center_i + half_p,
                   center_j - half_p:center_j + half_p]

    # 3. Range of search window.
    i_min = max(center_i - half_w, half_p)
    i_max = min(center_i + half_w, H - half_p)
    j_min = max(center_j - half_w, half_p)
    j_max = min(center_j + half_w, W - half_p)

    similar_patches = []

    # 4. Patch center.
    for i in range(i_min, i_max):
        for j in range(j_min, j_max):
            current_patch = im[i - half_p:i + half_p,
                               j - half_p:j + half_p]
            dist = distpatch(ref_patch.flatten(),
                             current_patch.flatten(),
                             khard=patch_size)
            if dist < threshard:
                similar_patches.append((dist, current_patch.copy()))

    # Use *ref_patch* if no similar patches are found.
    if len(similar_patches) == 0:
        similar_patches.append((0.0, ref_patch.copy()))

    # 5. Sorting, then keep the smallest *max_similar_patches* patches.
    similar_patches.sort(key=lambda x: x[0])
    selected = [p for (d, p) in similar_patches[:max_similar_patches]]

    # 6. 3D block: (patch_size, patch_size, nb_patches)
    block_3D = np.stack(selected, axis=2)

    return block_3D, (center_i, center_j)


def sigma_nd(coeffs, l=lamb2d, sigm =sigma):
    thresh = l * sigm
    coeffs = coeffs.copy()
    coeffs[np.abs(coeffs) < thresh] = 0
    return coeffs

def ondelet_3D(block_3D):
    block_3D = np.asarray(block_3D, dtype=np.float32)
    coeffs = dctn(block_3D, axes=(0, 1, 2), type=2, norm='ortho')
    return coeffs

def invondelet_3D(coeffs):
    coeffs = np.asarray(coeffs, dtype=np.float32)
    block_rec = idctn(coeffs, axes=(0, 1, 2), type=2, norm='ortho')
    return block_rec

def Step1_collaborative_filtering(block_3D):
    D = ondelet_3D(block_3D)          # 3D transform
    D_thr = sigma_nd(coeffs=D)  # 在频域 shrink
    patch_filtered = invondelet_3D(D_thr)
    return patch_filtered